In [3]:
from src import dino
from src.tone_mappers.bronto import bronto




input_file = "images/LuxoDoubleChecker.exr" # replace with your actual file of interest
image = dino.read_image(input_file)
tonemapped = bronto(image)
output_file = "_output/bronto_tonemapped.png"
dino.write_image(output_file, tonemapped)






    


